# Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/Proactiveware/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Proactiveware


In [ ]:
!pip install ultralytics
!pip install mpl-interactions

In [ ]:
!python -m pip install pyyaml==5.1
# Detectron2 has not released pre-built binaries for the latest pytorch (https://github.com/facebookresearch/detectron2/issues/4053)
# so we install from source instead. This takes a few minutes.
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

# Install pre-built detectron2 that matches pytorch version, if released:
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
#!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/{CUDA_VERSION}/{TORCH_VERSION}/index.html

# exit(0)  # After installation, you may need to "restart runtime" in Colab. This line can also restart runtime

# Lib

In [4]:
import detectron2
from detectron2.data import MetadataCatalog
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
import os
from detectron2.utils.visualizer import ColorMode
import glob
import cv2
import random
from google.colab.patches import cv2_imshow
from detectron2.utils.visualizer import Visualizer


import matplotlib.pyplot as plt
from ipywidgets import interact 
import plotly.express as px
from skimage import io
from mpl_interactions import ioff, panhandler, zoom_factory
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import numpy as np

from ultralytics import YOLO
import os
from PIL import Image
from detectron2.utils.visualizer import ColorMode
import glob
from detectron2.data.datasets import register_coco_instances

# %matplotlib widget
# %matplotlib inline


# Functions

In [29]:
def test_YOLO(file_name):

  model = YOLO("/content/drive/MyDrive/Proactiveware/runs/detect/train4/weights/best.pt")

  results = model.predict("data/folder_for_test_pic/new/{}".format(file_name))


  result = results[0]
  # print(result.boxes)
  output = []
  for box in result.boxes:
      x1, y1, x2, y2 = [
          round(x) for x in box.xyxy[0].tolist()
      ]
      class_id = box.cls[0].item()
      prob = round(box.conf[0].item(), 2)
      output.append([
          x1, y1, x2, y2, result.names[class_id], prob
      ])


  for box in result.boxes:
    class_id = result.names[box.cls[0].item()]
    cords = box.xyxy[0].tolist()
    cords = [round(x) for x in cords]
    conf = round(box.conf[0].item(), 2)
    print("Object type:", class_id)
    print("Coordinates:", cords)
    print("Probability:", conf)
    print("---")

  Image.fromarray(result.plot()[:,:,::-1])
  plt.imshow(Image.fromarray(result.plot()[:,:,::-1]), aspect=1.1)
  plt.show()


In [28]:

def test_detectron(file_name):
  cfg = get_cfg()
  cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))
  cfg.DATALOADER.NUM_WORKERS = 2
  cfg.OUTPUT_DIR = "output"
  cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3
  cfg.MODEL.WEIGHTS = "output/model_final_2.pth"
  cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set the testing threshold for this model
  cfg.DATASETS.TRAIN = ("my_dataset_train",)
  cfg.DATASETS.TEST = ("my_dataset_val",)
  cfg.MODEL.DEVICE='cpu'

  MetadataCatalog.get("my_dataset_train").set(thing_classes=['roof-panel', 'panel', 'roof'])


  metadata = MetadataCatalog.get(cfg.DATASETS.TRAIN[0])
  predictor = DefaultPredictor(cfg)


  imageName =  "data/folder_for_test_pic/new/{}".format(file_name)
  im = cv2.imread(imageName)
  outputs = predictor(im)

  v = Visualizer(im[:, :, ::-1],
                metadata, 
                scale=1.1
                )
  out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
  cv2_imshow(out.get_image()[:, :, ::-1])



# Data

In [18]:
# this is list of images for test the model
# note that model dosnt see this figure befire

dir_test_image = '/content/drive/MyDrive/Proactiveware/data/folder_for_test_pic/new'
list_of_figure = os.listdir(dir_test_image) 

list_of_figure.sort()

# Import trained models

In this part, we loaded the fine-tuned YOLOv8 and Fast R-CNN models on our dataset. We utilized these models to predict the presence of roofs and solar panels in new images that the models had not seen before.

**You can compare the result of two model**

## YOLO

In [30]:
#"The process may take a few seconds. Please wait."
interact(test_YOLO, file_name=list_of_figure)

interactive(children=(Dropdown(description='file_name', options=('new_01.png', 'new_02.png', 'new_03.png', 'ne…

<function __main__.test_YOLO(file_name)>

# Fast R-CNN

In [26]:
#"The process may take a few seconds. Please wait."
interact(test_detectron, file_name=list_of_figure)

interactive(children=(Dropdown(description='file_name', options=('new_01.png', 'new_02.png', 'new_03.png', 'ne…

<function __main__.test_detectron(file_name)>